In [52]:
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(
    base_url="http://host.docker.internal:11434", # env var
    model="qwen2.5:latest", # env var
    system="you are an expert AI agent",
    verbose=True
    )

In [49]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "password"

graph = Neo4jGraph()

In [32]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load example document
with open("J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt") as f:
    part1 = f.read()
with open("J. K. Rowling - Harry Potter 2 - The Chamber Of Secrets.txt") as f:
    part2 = f.read()
with open("J. K. Rowling - Harry Potter 3 - Prisoner of Azkaban.txt") as f:
    part3= f.read()
# with open("J. K. Rowling - Harry Potter 4 - The Goblet of Fire.txt") as f:
#     part4 = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=600,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.create_documents([part1, part2, part3])
print(texts[0])
print(texts[1])

page_content='Harry Potter and the Sorcerer's Stone


CHAPTER ONE

THE BOY WHO LIVED

Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say
that they were perfectly normal, thank you very much. They were the last
people you'd expect to be involved in anything strange or mysterious,
because they just didn't hold with such nonsense.'
page_content='Mr. Dursley was the director of a firm called Grunnings, which made
drills. He was a big, beefy man with hardly any neck, although he did
have a very large mustache. Mrs. Dursley was thin and blonde and had
nearly twice the usual amount of neck, which came in very useful as she
spent so much of her time craning over garden fences, spying on the
neighbors. The Dursleys had a small son called Dudley and in their
opinion there was no finer boy anywhere.'


In [57]:
from time import sleep
import sys

for i in range(len(texts)):
    try:
        llm_transformer_filtered = LLMGraphTransformer(
        llm=llm,
        allowed_nodes=["Person", "Organization"],
        allowed_relationships=[
            ("Person", "WORKED_WITH", "Person"),
            ("Person", "WORKED_IN", "Organization"),
            ("Person", "FRIEND", "Person"),
            ("Person", "SPOUSE", "Person"),
            ("Person", "OWNED", "Organization")
        ],
        strict_mode=False
        )
        documents = [texts[i]]
        print("before converting")
        graph_documents_filtered = llm_transformer_filtered.convert_to_graph_documents(
        documents
        )
        print("after converting")
        graph.add_graph_documents(graph_documents_filtered, include_source=True)
        print(f"Nodes:{graph_documents_filtered[0].nodes}")
        print(f"Relationships:{graph_documents_filtered[0].relationships}")
        # sys.stdout.write('\r')
        # sys.stdout.write("[%-20s] %d%%" % ('='*i, 5*i))
        # sys.stdout.flush()
    except Exception as e:
        print("EXCEPTION!")
        print(type(e))
        print(f"document:{[texts[i]]}")
        continue
    else:
        continue

before converting
after converting
Nodes:[]
Relationships:[]
before converting
after converting
Nodes:[Node(id='Grunnings', type='Organization', properties={}), Node(id='Mr. Dursley', type='Person', properties={}), Node(id='Mrs. Dursley', type='Person', properties={})]
Relationships:[Relationship(source=Node(id='Mr. Dursley', type='Person', properties={}), target=Node(id='Grunnings', type='Organization', properties={}), type='WORKED_IN', properties={}), Relationship(source=Node(id='Mrs. Dursley', type='Person', properties={}), target=Node(id='Mr. Dursley', type='Person', properties={}), type='SPOUSE', properties={})]
before converting
after converting
Nodes:[]
Relationships:[]
before converting
after converting
Nodes:[]
Relationships:[]
before converting
after converting
Nodes:[]
Relationships:[]
before converting
after converting
Nodes:[]
Relationships:[]
before converting
after converting
Nodes:[]
Relationships:[]
before converting
after converting
Nodes:[]
Relationships:[]
before co